# Import Libraries

In [37]:
import nltk
from nltk import bigrams,trigrams 
from nltk.corpus import reuters
from collections import Counter, defaultdict
from gensim.test.utils import datapath
from gensim.corpora import WikiCorpus

# Data-set

In [38]:
dataset_path = datapath('enwiki-latest-pages-articles1.xml-p000000010p000030302-shortened.bz2')
wiki_sentences = WikiCorpus(dataset_path).get_texts()

In [39]:
wiki_sentences

<generator object WikiCorpus.get_texts at 0x0000018A9B47DC48>


# get Reuters DataSet

In [40]:
nltk.download('punkt')
nltk.download('reuters')
reuters_sentences  = reuters.sents()

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package reuters to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package reuters is already up-to-date!


In [41]:
reuters_sentences

[['ASIAN', 'EXPORTERS', 'FEAR', 'DAMAGE', 'FROM', 'U', '.', 'S', '.-', 'JAPAN', 'RIFT', 'Mounting', 'trade', 'friction', 'between', 'the', 'U', '.', 'S', '.', 'And', 'Japan', 'has', 'raised', 'fears', 'among', 'many', 'of', 'Asia', "'", 's', 'exporting', 'nations', 'that', 'the', 'row', 'could', 'inflict', 'far', '-', 'reaching', 'economic', 'damage', ',', 'businessmen', 'and', 'officials', 'said', '.'], ['They', 'told', 'Reuter', 'correspondents', 'in', 'Asian', 'capitals', 'a', 'U', '.', 'S', '.', 'Move', 'against', 'Japan', 'might', 'boost', 'protectionist', 'sentiment', 'in', 'the', 'U', '.', 'S', '.', 'And', 'lead', 'to', 'curbs', 'on', 'American', 'imports', 'of', 'their', 'products', '.'], ...]

# Model

In [52]:
def calculate_probabilities(sentence_model):
  for next_word in sentence_model:
    next_words = sentence_model[next_word]
    total_word_count = float(sum(next_words.values()))
    for previous_word in next_words:
      next_words[previous_word]/=total_word_count

In [43]:
def calculate_sigle_word_probability(sentence_model,word_count):
  for word in sentence_model:
    sentence_model[word]/=word_count

In [44]:
def convert_to_lower(pa):
  if type(pa)==str:
    return pa.lower()
  return pa

In [45]:
sentence_model4 = defaultdict(lambda: set())
sentence_model5 = defaultdict(lambda: set())

def calculate_word_count(sentence_model1,sentence_model2,sentence_model3,sentences):
  word_count = 0
  for sentence in sentences:
    for word in sentence:
      word_count += 1
      sentence_model1[word] += 1
    for previous_word2, previous_word1, next_word in trigrams(sentence, pad_right=True, pad_left=True):
      previous_word1 = convert_to_lower(previous_word1)
      previous_word2 = convert_to_lower(previous_word2)
      next_word = convert_to_lower(next_word)
      sentence_model2[next_word][previous_word1] += 1
      sentence_model3[next_word][previous_word2] += 1
      sentence_model4[previous_word1].add(next_word)
      sentence_model5[previous_word2].add(next_word)

  return word_count

In [46]:
sentence_model1 = defaultdict(lambda:0)
sentence_model2 = defaultdict(lambda: defaultdict(lambda:0))
sentence_model3 = defaultdict(lambda: defaultdict(lambda:0))

In [47]:
wiki_word_count = calculate_word_count(sentence_model1,sentence_model2,sentence_model3,wiki_sentences)

In [48]:
wiki_word_count

452944

In [49]:
reuters_word_count = calculate_word_count(sentence_model1, sentence_model2, sentence_model3, reuters_sentences)

In [50]:
reuters_word_count

1720917

In [53]:
calculate_probabilities(sentence_model2)

In [54]:
calculate_probabilities(sentence_model3)

In [55]:
total_word = wiki_word_count + reuters_word_count
calculate_sigle_word_probability(sentence_model1, total_word)

In [56]:
max_probability_words = []

def make_word_suggestion_by_trigram(previous_word2, previous_word1):
  for next_word in sentence_model4[previous_word1] & sentence_model5[previous_word2]:
    naiveBias_trigram_weight = sentence_model1[next_word] * sentence_model2[next_word][previous_word1]*sentence_model3[next_word][previous_word2]
    max_probability_words.append((next_word, naiveBias_trigram_weight))

In [57]:
make_word_suggestion_by_trigram('my','name')
max_probability_words.sort(key=lambda o:o[1],reverse=True)

In [59]:
print(*max_probability_words[:10])

('is', 2.684929364088014e-08) ('to', 1.1753701280369164e-09) (',', 5.59438489695973e-10) ('in', 4.6278217257881707e-10) ('and', 4.5063509855288127e-10) ('for', 3.738790783749001e-10) ('would', 3.5383441884561533e-10) ('will', 2.8735045390861837e-10) ('that', 2.0690529880634479e-10) ('or', 1.711572663102755e-10)


In [ ]:
while(True):
    text = input("Enter your line: ")
    if text == "stop":
        print("The Program Stopped.....")
        break
    
    else:
        try:
            max_probability_words = []
            text = text.split(" ")
            make_word_suggestion_by_trigram(text[0],text[1])
            max_probability_words.sort(key=lambda o:o[1],reverse=True)
            print(*max_probability_words[:10])
            
        except:
            continue

Enter your line: go to
('the', 5.540810603048347e-07) ('go', 3.8299494667680125e-07) ('sell', 2.6532711001692346e-07) ('settle', 2.5319006137006924e-07) ('bring', 2.3749859383542032e-07) ('a', 1.2340368357350155e-07) ('do', 1.1895936962643927e-07) ('slow', 1.0666801915194969e-07) ('strike', 7.78095691160883e-08) ('10', 6.066503904930029e-08)
Enter your line: bangladesh is
('nominally', 6.571585895194902e-08) ('an', 2.7933336734722146e-08) ('no', 2.7441100252514623e-08) ('awarded', 9.018939567956929e-09) ('a', 7.322592118661084e-09) ('the', 3.647520655713172e-09) ('on', 2.234686395822939e-09) ('for', 1.6450679448495607e-09) ('cut', 8.565888110322454e-10) ('of', 5.069108054289822e-10)
Enter your line: i am
('afraid', 1.0350247784931974e-06) ('sure', 8.036310732474202e-07) ('astonished', 4.600110126636432e-07) ('speculating', 3.6800881013091455e-07) ('deeply', 1.8818632336239947e-07) ('convinced', 1.5333700422121436e-07) ('confident', 1.2893042223444673e-07) ('inclined', 4.600110126636432